In [1]:
# Uncomment to install dependencies
#!pip install pandas
#!pip install youtube_dl
#!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [2]:
import pandas as pd
import random
import json
import os
from youtube_dl import YoutubeDL

from google_auth_oauthlib.flow import InstalledAppFlow
from apiclient.discovery import build
from apiclient.http import MediaFileUpload
from apiclient import errors

yt_url_prefix = 'https://www.youtube.com/watch?v='
output_folder = 'outputs/'

In [3]:
# GOOGLE DRIVE API CONSTANTS
flow = InstalledAppFlow.from_client_secrets_file('credentials.json', ['https://www.googleapis.com/auth/drive'])
creds = flow.run_local_server(port=0)
folder_id = '1UnDIe4VHMM8bZzIfKEG8NJdQGnNrrxEG'
drive_api = build('drive', 'v3', credentials=creds)

# Helpful function
def get_file_list_from_folder(service, folder_id):
    """Print files belonging to a folder.

    Args:
    service: Drive API service instance.
    folder_id: ID of the folder to print files from.
    """
    kwargs = {
        "q": "'{}' in parents".format(folder_id)
    }
    request = service.files().list(**kwargs)
    while request is not None:
      response = request.execute()
      # Do stuff with response['files']
      request = service.files().list_next(request, response)
    return response

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=786879558582-eh0bho168rp2k1qord3av8r5u10q64sl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A49362%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=wEX4FGQFzWDNoYgN8S1BX48VTs4GMV&access_type=offline


In [4]:
trailers = pd.read_csv("trailers.csv", index_col=0)
#trailers

In [5]:
uploaded = get_file_list_from_folder(drive_api, folder_id)
uploaded = set([d['name'] for d in uploaded['files']])
uploaded

set()

In [ ]:
ydl_opts = {
    'outtmpl': output_folder + '%(id)s.%(ext)s',
    'format': 'bestvideo'
}
num_trailers_max = 5

with YoutubeDL(ydl_opts) as ydl:
    for t in trailers['youtube_key'][:num_trailers_max]:
        # Helpful Constants
        yt_link = yt_url_prefix + t
        file_name = t + ".mp4"
        
        if file_name not in uploaded:
            # Download Video
            print(f"Downloading {file_name}")
            trailer_yt_info = ydl.extract_info(yt_link, download=True)

            # Upload to Drive
            print(f"Uploading file {file_name}...")
            body = {'name': file_name, 'parents': [folder_id]}
            media = MediaFileUpload(output_folder + file_name)
            fiahl = drive_api.files().create(body=body, media_body=media).execute()
            print(f"Created file '{fiahl.get('name')}' id '{fiahl.get('id')}'.")

            # Delete video from hard drive
            os.remove(output_folder + file_name)
            print(f"Removed {file_name}")
            
            print()
        


[youtube] K_tLp7T6U1c: Downloading webpage
[youtube] K_tLp7T6U1c: Downloading embed webpage
[youtube] K_tLp7T6U1c: Refetching age-gated info webpage
[download] Destination: outputs/K_tLp7T6U1c.mp4
[download] 100% of 9.62MiB in 00:0419MiB/s ETA 00:009
Uploading file K_tLp7T6U1c.mp4...
